## Julia on Colab Instructions
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. If you need a GPU: _Runtime_ > _Change runtime type_ > _Harware accelerator_ = _GPU_.
3. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia and other packages (if needed, update `JULIA_VERSION` and the other parameters). This takes a couple of minutes.
4. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.

_Notes_:
* If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2, 3 and 4.
* After installation, if you want to change the Julia version or activate/deactivate the GPU, you will need to reset the Runtime: _Runtime_ > _Factory reset runtime_ and repeat steps 3 and 4.

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.6.1" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools https://github.com/Dale-Black/DistanceTransforms.jl.git ImageMorphology"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=40
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Success! Please reload this page and jump to the next section."
fi

Installing Julia 1.6.1 on the current Colab Runtime...
2021-06-28 13:11:34 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.6/julia-1.6.1-linux-x86_64.tar.gz [112784227/112784227] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package BenchmarkTools...
Installing Julia package https://github.com/Dale-Black/DistanceTransforms.jl.git...
Installing Julia package ImageMorphology...
Installing IJulia kernel...
[ Info: Installing julia kernelspec in /root/.local/share/jupyter/kernels/julia-1.6

Success! Please reload this page and jump to the next section.


## Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system:

In [1]:
versioninfo()

Julia Version 1.6.1
Commit 6aaedecc44 (2021-04-23 05:59 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) CPU @ 2.30GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, haswell)
Environment:
  JULIA_NUM_THREADS = 40


## Import Packages

In [2]:
using BenchmarkTools
using DistanceTransforms
using ImageMorphology

## Load test data

In [3]:
x = rand([0,1], (112, 112, 96, 1, 4));

## Pure feature transform

In [4]:
x_ft = ImageMorphology.feature_transform(.!(Bool.(x)));

In [5]:
@benchmark ImageMorphology.feature_transform(.!(Bool.($x)))

BenchmarkTools.Trial: 
  memory estimate:  1.03 GiB
  allocs estimate:  23124023
  --------------
  minimum time:     2.311 s (7.10% GC)
  median time:      2.444 s (7.40% GC)
  mean time:        2.422 s (8.93% GC)
  maximum time:     2.510 s (12.11% GC)
  --------------
  samples:          3
  evals/sample:     1

## Pure distance transform

In [6]:
x_dt = ImageMorphology.distance_transform(x_ft);

In [7]:
@benchmark ImageMorphology.distance_transform($x_ft)

BenchmarkTools.Trial: 
  memory estimate:  36.75 MiB
  allocs estimate:  3
  --------------
  minimum time:     54.824 ms (0.00% GC)
  median time:      56.449 ms (0.00% GC)
  mean time:        57.384 ms (1.89% GC)
  maximum time:     66.064 ms (11.19% GC)
  --------------
  samples:          88
  evals/sample:     1

## Full `compute_dtm`

In [8]:
x_dtm = DistanceTransforms.compute_dtm(x);

In [9]:
@benchmark DistanceTransforms.compute_dtm($x)

BenchmarkTools.Trial: 
  memory estimate:  1.07 GiB
  allocs estimate:  23124026
  --------------
  minimum time:     2.324 s (5.82% GC)
  median time:      2.430 s (7.96% GC)
  mean time:        2.524 s (11.63% GC)
  maximum time:     2.818 s (19.60% GC)
  --------------
  samples:          3
  evals/sample:     1

## Time `chamfer_distance`

In [10]:
function chamfer_distance5D(x)
	dt = zeros(Int32, size(x))
	for batch in 1:size(x)[5]
		for channel in 1:size(x)[4]
			dt[:, :, :, channel, batch] = chamfer_distance3D(x[:, :, :, channel, batch])
		end
	end
	return dt
end

chamfer_distance5D (generic function with 1 method)

In [11]:
x_chmf = chamfer_distance5D(x);

In [12]:
@benchmark chamfer_distance5D(x)

BenchmarkTools.Trial: 
  memory estimate:  128.68 MiB
  allocs estimate:  1559
  --------------
  minimum time:     130.572 ms (0.00% GC)
  median time:      147.091 ms (9.63% GC)
  mean time:        143.841 ms (7.14% GC)
  maximum time:     161.384 ms (17.51% GC)
  --------------
  samples:          35
  evals/sample:     1